In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


### Exercise 1.1

In [ ]:
df = pd.read_stata('assignment8.dta')

df.head()
df[['state','wage_st', 'time']].describe(include='all')

,state,wage_st,time
count,820.000000,779.000000,820.000000
mean,0.807317,4.805712,0.500000
std,0.394647,0.358395,0.500305
min,0.000000,4.250000,0.000000
25%,1.000000,4.500000,0.000000
50%,1.000000,5.000000,0.500000
75%,1.000000,5.050000,1.000000
max,1.000000,6.250000,1.000000


In [ ]:
state_map = {1:"NJ", 0:"PA"}
df['state_name'] = df['state'].map(state_map)

df['state_name'].value_counts()

state_name
NJ    662
PA    158
Name: count, dtype: int64

In [ ]:
bins = np.arange(4.25, 5.65 + 0.10, 0.10)

bin_labels = [f"{b:.2f}" for b in bins[:-1]]
bin_labels[:5], bin_labels[-3:] #check

(['4.25', '4.35', '4.45', '4.55', '4.65'], ['5.35', '5.45', '5.55'])

In [ ]:
df_nonmissing = df.dropna(subset=['wage_st']).copy() #copy to not delete data

df_nonmissing['wage_bin'] = pd.cut(
    df_nonmissing['wage_st'],
    bins=bins, labels=bin_labels, right=False) #include left edge, exclude right edge

df_nonmissing['wage_bin'].value_counts().sort_index()

counts = (df_nonmissing.groupby(['state_name', 'time', 'wage_bin'])
          .size().rename('n').reset_index())

counts.head()